In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [16]:
# device configuration
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

print(f'Using {device} device')

Using mps device


In [17]:
# download MNIST dataset in local system
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor()
)

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [40]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [38]:
def train(batch_size, learning_rate, epochs):
    train_dataloader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for i in range(epochs):
        print(f'Epoch {i+1}\n-------------------------------')
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)

    print('Done!')

In [27]:
model = NeuralNetwork()

loss_fn = nn.CrossEntropyLoss()

In [28]:
batch_size = 100
learning_rate = 1e-3
epochs = 5

In [41]:
train(batch_size, learning_rate, epochs)

Epoch 1
-------------------------------
loss: 2.310167  [   64/60000]
loss: 2.296788  [ 6464/60000]
loss: 2.293979  [12864/60000]
loss: 2.295336  [19264/60000]
loss: 2.286585  [25664/60000]
loss: 2.278621  [32064/60000]
loss: 2.272911  [38464/60000]
loss: 2.267469  [44864/60000]
loss: 2.266604  [51264/60000]
loss: 2.265316  [57664/60000]
Test Error: 
 Accuracy: 36.4%, Avg loss: 2.258195 

Epoch 2
-------------------------------
loss: 2.252831  [   64/60000]
loss: 2.250501  [ 6464/60000]
loss: 2.251207  [12864/60000]
loss: 2.241937  [19264/60000]
loss: 2.244331  [25664/60000]
loss: 2.231754  [32064/60000]
loss: 2.223207  [38464/60000]
loss: 2.233521  [44864/60000]
loss: 2.199232  [51264/60000]
loss: 2.205567  [57664/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 2.192003 

Epoch 3
-------------------------------
loss: 2.211522  [   64/60000]
loss: 2.186722  [ 6464/60000]
loss: 2.177796  [12864/60000]
loss: 2.150613  [19264/60000]
loss: 2.174925  [25664/60000]
loss: 2.134375  [32064/600